### Result was published in the paper: Can Readability Enhance Recommendations on Community Question Answering Sites?
In proceedings of the 11th ACM Conference on Recommender Systems, August 2017 
Authors: Oghenemaro Anuyah, Ion Madrazo Azpiazu, David McNeill, Sole Pera 

We used the Yahoo! L16 webscope dataset 

## Import useful libraries

In [1]:
import pandas as pd
import nltk
import operator
import collections
import random                                                                                                                                   
import os    
import math
import sys
import random
import nltk.data                                                                                                                                
import string    
import collections
import collections as c                                                                                                            
import tflearn                                                                                  
import tensorflow as tf from scipy.io.arff import loadarff           
import json                                                                                                                                      
import scipy as sp                                                                                                                              
import numpy as np
import matplotlib.pyplot as plt 

from sklearn import linear_model
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
from scipy.spatial import distance
from sklearn import linear_model
from __future__ import division, print_function, absolute_import  
from tflearn.data_utils import to_categorical, pad_sequences, VocabularyProcessor 
from sklearn import datasets, svm, metrics                                                                                                      
from sklearn.naive_bayes import MultinomialNB                                                                                                   
from sklearn.ensemble import RandomForestClassifier                                                                                             
from sklearn.neighbors import KNeighborsClassifier                                                                                              
from sklearn.neural_network import MLPClassifier                                                                                               
from sklearn.tree import DecisionTreeClassifier       
from tflearn.datasets import imdb    
from scipy.stats import pearsonr                                                                                                                                 
from sys import argvfrom tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.contrib import learnimport string
from textstat.textstat import textstat
from collections import OrderedDict
from operator import itemgetter
from functools import reduce  
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')  

### Read the data

In [2]:
data= pd.read_csv('data/L16/ydata-yanswers-question-answer-ratings-v1_0.txt',names=['query_id','queries','question_id','query_ques_match','query_ans_sat','question_subject','question_content','answer','question_category_id','question_category_name'], delimiter='|')

In [3]:
data=data[['query_id','queries','question_subject','question_content','answer','query_ans_sat']] #select only the relevant columns

In [4]:
data[:2]

,query_id,queries,question_subject,question_content,answer,query_ans_sat
0,341,18 kids counting religion,what religion is the duggar family? 18 kids a...,\t,Evangelical or Baptist I believe.,1.666667
1,910,1949 slang,1949 to 1950 often used slang words?,I would like to know some basic slang words fr...,Keen\tHep\tSpiffy\tGam \tGat\tCat's pajamas\tC...,1.000000


In [5]:
len(data) ##check the length of the data

1571

### Function for the Content Similarity 

- This code is based on a paper by li et al.

In [6]:
ALPHA = 0.2
BETA = 0.45
ETA = 0.4
PHI = 0.2
DELTA = 0.85

brown_freqs = dict()
N = 0

In [7]:
def get_best_synset_pair(word_1, word_2):
    """ 
    Choose the pair with highest path similarity among all pairs. 
    Mimics pattern-seeking behavior of humans.
    """
    max_sim = -1.0
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    if len(synsets_1) == 0 or len(synsets_2) == 0:
        return None, None
    else:
        max_sim = -1.0
        best_pair = None, None
        for synset_1 in synsets_1:
            for synset_2 in synsets_2:
                sim = wn.path_similarity(synset_1, synset_2)
                if sim == None:
                    sim = 0
                if sim > max_sim:
                    max_sim = sim
                    best_pair = synset_1, synset_2
        return best_pair

In [8]:
def length_dist(synset_1, synset_2):
    """
    Return a measure of the length of the shortest path in the semantic 
    ontology (Wordnet in our case as well as the paper's) between two 
    synsets.
    """
    l_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return 0.0
    if synset_1 == synset_2:
        # if synset_1 and synset_2 are the same synset return 0
        l_dist = 0.0
    else:
        wset_1 = set([str(x.name()) for x in synset_1.lemmas()])        
        wset_2 = set([str(x.name()) for x in synset_2.lemmas()])
        if len(wset_1.intersection(wset_2)) > 0:
            # if synset_1 != synset_2 but there is word overlap, return 1.0
            l_dist = 1.0
        else:
            # just compute the shortest path between the two
            l_dist = synset_1.shortest_path_distance(synset_2)
            if l_dist is None:
                l_dist = 0.0
    # normalize path length to the range [0,1]
    return math.exp(-ALPHA * l_dist)

In [9]:
def hierarchy_dist(synset_1, synset_2):
    """
    Return a measure of depth in the ontology to model the fact that 
    nodes closer to the root are broader and have less semantic similarity
    than nodes further away from the root.
    """
    h_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return h_dist
    if synset_1 == synset_2:
        # return the depth of one of synset_1 or synset_2
        h_dist = max([x[1] for x in synset_1.hypernym_distances()])
    else:
        # find the max depth of least common subsumer
        hypernyms_1 = {x[0]:x[1] for x in synset_1.hypernym_distances()}
        hypernyms_2 = {x[0]:x[1] for x in synset_2.hypernym_distances()}
        lcs_candidates = set(hypernyms_1.keys()).intersection(
            set(hypernyms_2.keys()))
        if len(lcs_candidates) > 0:
            lcs_dists = []
            for lcs_candidate in lcs_candidates:
                lcs_d1 = 0
                if lcs_candidate in hypernyms_1:
                    lcs_d1 = hypernyms_1[lcs_candidate]
                lcs_d2 = 0
                if lcs_candidate in hypernyms_2:
                    lcs_d2 = hypernyms_2[lcs_candidate]
                lcs_dists.append(max([lcs_d1, lcs_d2]))
            h_dist = max(lcs_dists)
        else:
            h_dist = 0
    return ((math.exp(BETA * h_dist) - math.exp(-BETA * h_dist)) / 
        (math.exp(BETA * h_dist) + math.exp(-BETA * h_dist)))

In [10]:
def word_similarity(word_1, word_2):
    synset_pair = get_best_synset_pair(word_1, word_2)
    return (length_dist(synset_pair[0], synset_pair[1]) * 
        hierarchy_dist(synset_pair[0], synset_pair[1]))

In [11]:
def most_similar_word(word, word_set):
    """
    Find the word in the joint word set that is most similar to the word
    passed in. We use the algorithm above to compute word similarity between
    the word and each word in the joint word set, and return the most similar
    word and the actual similarity value.
    """
    max_sim = -1.0
    sim_word = ""
    for ref_word in word_set:
        sim = word_similarity(word, ref_word)
        if sim > max_sim:
            max_sim = sim
            sim_word = ref_word
    return sim_word, max_sim

In [12]:
def info_content(lookup_word):
    """
    Uses the Brown corpus available in NLTK to calculate a Laplace
    smoothed frequency distribution of words, then uses this information
    to compute the information content of the lookup_word.
    """
    global N
    if N == 0:
        # poor man's lazy evaluation
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if word not in brown_freqs:
                    brown_freqs[word] = 0
                brown_freqs[word] = brown_freqs[word] + 1
                N = N + 1
    lookup_word = lookup_word.lower()
    n = 0 if lookup_word not in brown_freqs else brown_freqs[lookup_word]
    return 1.0 - (math.log(n + 1) / math.log(N + 1))

In [13]:
def semantic_vector(words, joint_words, info_content_norm):
    """
    Computes the semantic vector of a sentence. The sentence is passed in as
    a collection of words. The size of the semantic vector is the same as the
    size of the joint word set. The elements are 1 if a word in the sentence
    already exists in the joint word set, or the similarity of the word to the
    most similar word in the joint word set if it doesn't. Both values are 
    further normalized by the word's (and similar word's) information content
    if info_content_norm is True.
    """
    sent_set = set(words)
    semvec = np.zeros(len(joint_words))
    i = 0
    for joint_word in joint_words:
        if joint_word in sent_set:
            # if word in union exists in the sentence, s(i) = 1 (unnormalized)
            semvec[i] = 1.0
            if info_content_norm:
                semvec[i] = semvec[i] * math.pow(info_content(joint_word), 2)
        else:
            # find the most similar word in the joint set and set the sim value
            sim_word, max_sim = most_similar_word(joint_word, sent_set)
            semvec[i] = PHI if max_sim > PHI else 0.0
            if info_content_norm:
                semvec[i] = semvec[i] * info_content(joint_word) * info_content(sim_word)
        i = i + 1
    return semvec

In [14]:
def semantic_similarity(sentence_1, sentence_2, info_content_norm):
    """
    Computes the semantic similarity between two sentences as the cosine
    similarity between the semantic vectors computed for each sentence.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = set(words_1).union(set(words_2))
    vec_1 = semantic_vector(words_1, joint_words, info_content_norm)
    vec_2 = semantic_vector(words_2, joint_words, info_content_norm)
    return np.dot(vec_1, vec_2.T) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))

In [15]:
def word_order_vector(words, joint_words, windex):
    """
    Computes the word order vector for a sentence. The sentence is passed
    in as a collection of words. The size of the word order vector is the
    same as the size of the joint word set. The elements of the word order
    vector are the position mapping (from the windex dictionary) of the 
    word in the joint set if the word exists in the sentence. If the word
    does not exist in the sentence, then the value of the element is the 
    position of the most similar word in the sentence as long as the similarity
    is above the threshold ETA.
    """
    wovec = np.zeros(len(joint_words))
    i = 0
    wordset = set(words)
    for joint_word in joint_words:
        if joint_word in wordset:
            # word in joint_words found in sentence, just populate the index
            wovec[i] = windex[joint_word]
        else:
            # word not in joint_words, find most similar word and populate
            # word_vector with the thresholded similarity
            sim_word, max_sim = most_similar_word(joint_word, wordset)
            if max_sim > ETA:
                wovec[i] = windex[sim_word]
            else:
                wovec[i] = 0
        i = i + 1
    return wovec

In [16]:
def word_order_similarity(sentence_1, sentence_2):
    """
    Computes the word-order similarity between two sentences as the normalized
    difference of word order between the two sentences.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = list(set(words_1).union(set(words_2)))
    windex = {x[1]: x[0] for x in enumerate(joint_words)}
    r1 = word_order_vector(words_1, joint_words, windex)
    r2 = word_order_vector(words_2, joint_words, windex)
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))

In [17]:
def similarity(sentence_1, sentence_2, info_content_norm):
    """
    Calculate the semantic similarity between two sentences. The last 
    parameter is True or False depending on whether information content
    normalization is desired or not.
    """
    return DELTA * semantic_similarity(sentence_1, sentence_2, info_content_norm) + \
        (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2)

## Using Read2vec readability

- Removed code (Yet to be released by author. Send email to git owner for access)

In [39]:
#requires an array of words
#not symbols, just numebrs and letters
#max sentence lenght == 20, words after the 20th word will be ignored
#readability is normalzied between 0 and 1. 0 means harder 1 simpler
def getReadabilityForSentence(words = []):
    wordIds = list(vocabProcessor.transform([" ".join(words)]))
    return sess.run(ypred, feed_dict={x: wordIds, keep_prob:1.0})[0][1]

In [40]:
getReadabilityForSentence(words = ["the","door","is","closed"])

0.91219229

In [41]:
getReadabilityForSentence(words=["accomodate","the","beautification"])

0.79620558

In [42]:
getReadabilityForSentence(words="she is a beautiful girl".split())

0.99997926

In [43]:
getReadabilityForSentence(words=["the","door","is","closed"])

0.91219229

In [44]:
getReadabilityForSentence(words="the presidential parliament has just concluded the elections however the president himself will not condone injustice".split())

0.00031201795

In [45]:
def Read2vec_sentences(sent=[]):   
    scores=[]
    sentences=[]
    lid=[]
    split_sentences=(tokenizer.tokenize(str(sent).replace("\\n"," ")))
    sentences= [s.lower().split() for s in split_sentences]          
    for s in sentences:
        scores.append(getReadabilityForSentence(words=s))
    n_s=sorted(scores, reverse=False)   
    return np.median(n_s) #use the median of the readability scores to calculate the overall readability of the sentence

In [46]:
Read2vec_sentences('The honorable president has arrived. He just spoke about how appreciative he is about effort to eradicate poverty.  she is good. she is fine. she is a girl. she is. commotion accomodability beauticonability')

0.9950276

In [47]:
#data['query_answer_sim']=[semantic_similarity(x, y,True) for x, y in zip(data.queries, data.answer)]

data=data.rename(columns={"ques_ans_sim":"query_answer_sim"})

In [48]:
simple_words=pd.read_csv('data/simple_words.txt', names=['words'], delimiter=('\n \t'), engine='python') #for dale-challe

In [49]:
simple_words=[s for s in simple_words.words]

In [50]:
len(simple_words)

2950

## Flesch Readability Formula

In [51]:
def syllables(word):
    count = 0
    vowels = 'aeiou'    
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le'):
            count+=1
        if word.endswith('ie'):
            count+=1
        if count == 0:
            count +=1
    return count

In [52]:
def Flesch_readability(text=[]):    
    split_sentences= tokenizer.tokenize(str(text).replace("\\n"," "))
    tokens=[x.split() for x in split_sentences]
    for w in tokens:
        cleaned_words = [[''.join(c for c in s if c not in string.punctuation) for s in m] for m in tokens]
        cleaned_words = [[s.lower() for s in k if s] for k in cleaned_words]
        cleaned_words=[[w for w in k if w.isalpha()] for k in cleaned_words]
    words=[a for m in cleaned_words for a in m ]    
    try:
        Total_words=len(words)
        Total_sentences=len(cleaned_words)    
        Total_Syllable=0
        for w in words:
            Total_Syllable+=syllables(w)
        A=(Total_words)/(Total_sentences)
        B=(Total_Syllable)/(Total_words)
    except Exception:
        A= 1
        B= 1
        pass
    else:
        pass    
    flesch_kincaid_difficulty=(206.835-(1.015*A)-(84.6*B))
    flesch_kincaid_grade_level=(0.39*A)+(11.8*B)-15.59
    if flesch_kincaid_difficulty>100:
        flesch_kincaid_difficulty=100
    if flesch_kincaid_difficulty<0:
        flesch_kincaid_difficulty=1
    return flesch_kincaid_difficulty  

In [53]:
Flesch_readability("Proposed a method that could produce a more distinct and content based relevance assessment. The methods are Reconciled Relevance ranking and reconciled relevance. In all, this paper is about ranking the documents retrieved in response to children's queries. It is important that we re-rank the documents (part of the filtering), after filtering the documents.)")

64.66583333333335

In [54]:
print (textstat.flesch_reading_ease('a sentence'))

86.37


In [55]:
data['flesch_query']= [textstat.flesch_reading_ease(x) for x in (data.queries)]
data['flesch_answer']= [textstat.flesch_reading_ease(x) for x in (data.answer)]

In [58]:
scorex1=[x for x in data.flesch_query]

new_s1=[]
for s in scorex1:
    if s<0:
        sc1=0
    elif s>100:
        sc1=100
    else:
        sc1=s
    new_s1.append(sc1)

In [59]:
scorex=[x for x in data.flesch_answer]

new_s2=[]
for s in scorex:
    if s<0:
        sc=0
    elif s>100:
        sc=100
    else:
        sc=s
    new_s2.append(sc)

In [60]:
data['flesch_answer']= new_s2
data['flesch_query']= new_s1

In [61]:
len(scorex)

1571

In [62]:
data[:10]

,query_id,queries,question_subject,question_content,answer,query_ans_sat,flesch_query,flesch_answer
0,341,18 kids counting religion,what religion is the duggar family? 18 kids a...,\t,Evangelical or Baptist I believe.,1.666667,84.34,32.56
1,910,1949 slang,1949 to 1950 often used slang words?,I would like to know some basic slang words fr...,Keen\tHep\tSpiffy\tGam \tGat\tCat's pajamas\tC...,1.000000,100.00,96.18
2,607,1950 pfennig,How much is this German 10 Pfennig Swastika co...,"So, I was looking through my foreign coins... ...",The 10 Reich-pfennig is made of zinc so that i...,1.428571,100.00,89.99
3,593,2000 cc is how many liters,whAT DOES IT MEAN MOTOR CAPCITY 2000 CC OR 150...,\t,It means the amount of empty space (how much l...,2.571429,100.00,83.66
4,593,2000 cc is how many liters,How many grams are in 2 liters (of water)?,?\t,1 cc of water = 1 gram\t\t2 liters = 2000 cc =...,1.428571,100.00,100.00
5,593,2000 cc is how many liters,Toyota Camry Stationwagon: engine CC and does ...,"I know nothing about car engines, and I need t...","Only Mazda offered rotary engines, Toyota did ...",2.857143,100.00,71.31
6,593,2000 cc is how many liters,I need help on my h.w. in math. its a 6th grad...,"A tank that is 25 cm long, 10 cm wide, and 8 c...","Firstly, calculate the capacity of the tank in...",1.285714,100.00,100.00
7,525,7 point gpa scale,how to convert gpa to 4.0 scale?>?,im having a problem converting my gpa from the...,Your guidence counselor has a sheet to give yo...,1.714286,100.00,77.94
8,525,7 point gpa scale,"On a 7 point scale, my GPA is 6.4. What is it ...",\t,"aroun 3.8, but ill calculate the exact for you...",1.571429,100.00,100.00
9,525,7 point gpa scale,How do you calculate gpa on a 7 point scale (a...,\tok well thats the way my school does it\tk (...,This doesn't make any sense.\t\tthere's only 5...,2.571429,100.00,100.00


In [63]:
data['Read2vec_query']= [Read2vec_sentences(x) for x in (data.queries)]
data['Read2vec_answer']= [Read2vec_sentences(x) for x in (data.answer)]

In [64]:
#del data['read2vec_dist']

In [65]:
data['flesch_dist']=[distance.euclidean(x,y) for x,y in zip (data.flesch_query, data.flesch_answer)]
data['read2vec_dist']=[distance.euclidean(x,y) for x,y in zip (data.Read2vec_query, data.Read2vec_answer)]

In [66]:
#data = data.rename(columns={'q_a_flesch_dist': 'flesch_dist', 'q_a_read2vec_dist': 'read2vec_dist'})

In [67]:
data['Relevance_score'] = data.query_ans_sat

In [68]:
values= [x for x in data.Relevance_score]

In [69]:
new_v=[]
for x in values:
    if x >=1 and x < 1.5:
        v=3
    elif x >= 1.5 and x < 2.5:
        v=2
    else:
        v=1
    new_v.append(v)

In [70]:
data.Relevance_score=new_v

In [71]:
data[:5]

,query_id,queries,question_subject,question_content,answer,query_ans_sat,flesch_query,flesch_answer,Read2vec_query,Read2vec_answer,flesch_dist,read2vec_dist,Relevance_score
0,341,18 kids counting religion,what religion is the duggar family? 18 kids a...,\t,Evangelical or Baptist I believe.,1.666667,84.34,32.56,0.200418,0.929365,51.78,0.728947,2
1,910,1949 slang,1949 to 1950 often used slang words?,I would like to know some basic slang words fr...,Keen\tHep\tSpiffy\tGam \tGat\tCat's pajamas\tC...,1.000000,100.00,96.18,0.764292,0.555969,3.82,0.208323,3
2,607,1950 pfennig,How much is this German 10 Pfennig Swastika co...,"So, I was looking through my foreign coins... ...",The 10 Reich-pfennig is made of zinc so that i...,1.428571,100.00,89.99,0.805796,0.999999,10.01,0.194203,3
3,593,2000 cc is how many liters,whAT DOES IT MEAN MOTOR CAPCITY 2000 CC OR 150...,\t,It means the amount of empty space (how much l...,2.571429,100.00,83.66,0.417989,0.088361,16.34,0.329628,1
4,593,2000 cc is how many liters,How many grams are in 2 liters (of water)?,?\t,1 cc of water = 1 gram\t\t2 liters = 2000 cc =...,1.428571,100.00,100.00,0.417989,0.999997,0.00,0.582009,3


In [72]:
all_ids=[k for k in data.query_id] #get all query ids

In [73]:
from collections import Counter

cnt= Counter(all_ids)
new_ids= [k for k,v in cnt.items() if v>1] #saves query ids that have more than one answer to the question
other_ids= [k for k,v in cnt.items() if v==1]

In [74]:
len(other_ids)

245

In [75]:
train_data= data[data.query_id.isin(other_ids)]

In [76]:
train_data['query_answer_sim']=[semantic_similarity(x, y,True) for x, y in zip(train_data.queries, train_data.answer)]

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [1261]:
train_data['query_question_sim']=[semantic_similarity(x, y,True) for x, y in zip(train_data.queries, train_data.question_content)]

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in double_scalars
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [78]:
test_data= data[data.query_id.isin(new_ids)]

In [79]:
#new_data['query_question_sim']=[semantic_similarity(x, y,True) for x, y in zip(new_data.queries, new_data.question_content)]

In [ ]:
test_data['query_answer_sim']=[semantic_similarity(x, y,True) for x, y in zip(test_data.queries, test_data.answer)]

In [1262]:
test_data['query_question_sim']=[semantic_similarity(x, y,True) for x, y in zip(test_data.queries, test_data.question_content)]

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in double_scalars
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [1267]:
test_data[:3]

,query_id,queries,question_subject,question_content,answer,query_ans_sat,flesch_query,flesch_answer,Read2vec_query,Read2vec_answer,flesch_dist,read2vec_dist,Relevance_score,query_answer_sim,query_question_sim
3,593,2000 cc is how many liters,whAT DOES IT MEAN MOTOR CAPCITY 2000 CC OR 150...,\t,It means the amount of empty space (how much l...,2.571429,100.0,83.66,0.417989,0.088361,16.34,0.329628,1,0.131272,0.000000
4,593,2000 cc is how many liters,How many grams are in 2 liters (of water)?,?\t,1 cc of water = 1 gram\t\t2 liters = 2000 cc =...,1.428571,100.0,100.00,0.417989,0.999997,0.00,0.582009,3,0.735512,0.000000
5,593,2000 cc is how many liters,Toyota Camry Stationwagon: engine CC and does ...,"I know nothing about car engines, and I need t...","Only Mazda offered rotary engines, Toyota did ...",2.857143,100.0,71.31,0.417989,0.517005,28.69,0.099016,1,0.030845,0.198355


In [1268]:
train_data=train_data.replace([np.nan, -np.inf, np.inf], 0)

In [85]:
max(train_data.Relevance_score)

3

In [86]:
def random_rec(q_id=''):    
    query_id=int(q_id)   
    recs=list(set([s for s in test_data['answer'][test_data.query_id==query_id]]))
    random.shuffle(recs)
    return recs      

In [1452]:
def similarity_rec(q_id=''):    
    query_id=int(q_id)    
    own= test_data[test_data['query_id']==query_id]
    own_rank= own.sort_values('query_question_sim', ascending=False)
    sem=[s for s in own_rank.answer]
    return sem

In [1453]:
similarity_rec('593')

['Firstly, calculate the capacity of the tank in cubic cm.\t\t25 x 10 x 8 = 2,000 cubic cm\t\tThen convert cubic cm (or "cc") to liters: 1,000 cc = 1 litre\t\tTherefore the tank holds 2 liters of water.\t\t–––––––––––––––––––––––––––',
 'Only Mazda offered rotary engines, Toyota did not. Mazda still has the RX-8 with a rotary engine.',
 'It means the amount of empty space (how much liquid it would hold) in a motorcycle engine when the piston is at the bottom of the stroke.  For instance a chevy 350 Cubic  Inch motor holds 350 cubic inches of liquid.  To compare - a 1600cc holds approx. 95 Cubic Inches of liquid.',
 '1 cc of water = 1 gram\t\t2 liters = 2000 cc = 2000 grams = 2 Kg']

In [89]:
def Flesch_rec(q_id=''):    
    query_id=int(q_id)
    #readability= Flesch_readability
    own= test_data[test_data['query_id']==query_id]
    own_rank= own.sort_values('flesch_answer', ascending=False)
    sem=[s for s in own_rank.answer]
    return(sem)

In [90]:
def Read2vec_rec(q_id=''):    
    query_id=int(q_id)
    #readability= Read2vec_readability
    own= test_data[test_data['query_id']==query_id]
    own_rank= own.sort_values('Read2vec_answer', ascending=False)
    sem=[s for s in own_rank.answer]
    return sem

In [91]:
def Read2vec_rec_inv(q_id=''):    
    query_id=int(q_id)
    #readability= Inverse of Read2vec_readability
    own= test_data[test_data['query_id']==query_id]
    own_rank= own.sort_values('Read2vec_answer', ascending=True)
    sem=[s for s in own_rank.answer]
    return sem

In [92]:
Read2vec_rec_inv('593')

['It means the amount of empty space (how much liquid it would hold) in a motorcycle engine when the piston is at the bottom of the stroke.  For instance a chevy 350 Cubic  Inch motor holds 350 cubic inches of liquid.  To compare - a 1600cc holds approx. 95 Cubic Inches of liquid.',
 'Only Mazda offered rotary engines, Toyota did not. Mazda still has the RX-8 with a rotary engine.',
 'Firstly, calculate the capacity of the tank in cubic cm.\t\t25 x 10 x 8 = 2,000 cubic cm\t\tThen convert cubic cm (or "cc") to liters: 1,000 cc = 1 litre\t\tTherefore the tank holds 2 liters of water.\t\t–––––––––––––––––––––––––––',
 '1 cc of water = 1 gram\t\t2 liters = 2000 cc = 2000 grams = 2 Kg']

In [93]:
def Flesch_q_a_distance(q_id=''):
    sem_sim=[]
    values={}
    query_id=int(q_id)
    #readability= Flesch_readability
    own= test_data[test_data['query_id']==query_id]
    own_rank= own.sort_values('flesch_dist', ascending=True)
    sem=[s for s in own_rank.answer]
    return sem

In [94]:
def Read2vec_q_a_distance(q_id=''):    
    query_id=int(q_id)
    #readability= Flesch_readability
    own= test_data[test_data['query_id']==query_id]
    own_rank= own.sort_values('read2vec_dist', ascending=True)
    sem=[s for s in own_rank.answer]
    return sem

In [95]:
Read2vec_q_a_distance('593')

['Only Mazda offered rotary engines, Toyota did not. Mazda still has the RX-8 with a rotary engine.',
 'Firstly, calculate the capacity of the tank in cubic cm.\t\t25 x 10 x 8 = 2,000 cubic cm\t\tThen convert cubic cm (or "cc") to liters: 1,000 cc = 1 litre\t\tTherefore the tank holds 2 liters of water.\t\t–––––––––––––––––––––––––––',
 'It means the amount of empty space (how much liquid it would hold) in a motorcycle engine when the piston is at the bottom of the stroke.  For instance a chevy 350 Cubic  Inch motor holds 350 cubic inches of liquid.  To compare - a 1600cc holds approx. 95 Cubic Inches of liquid.',
 '1 cc of water = 1 gram\t\t2 liters = 2000 cc = 2000 grams = 2 Kg']

In [96]:
def Get_ranks(q_id='', func=''):
    que_id=int(q_id)
    Recs=func(que_id)
    own= test_data[test_data['query_id']==que_id]
    ideal= own.sort_values('query_ans_sat', ascending=True)
    ideal=ideal['answer'][:1]
    ideal=' '.join(s for s in ideal)
    
    position= Recs.index(ideal)+1
    return position

In [97]:
Get_ranks('593', func=random_rec)

1

In [98]:
baselines_mrr=pd.DataFrame(columns=('query_id','random_mrr','similarity_mrr','flesch_mrr','Read2vec_mrr','flesch_dist_mrr','Read2vec_dist_mrr'))

In [99]:
baselines_mrr.query_id=new_ids

In [100]:
baselines_mrr.random_mrr=[1/(Get_ranks(x, func=random_rec)) for x in baselines_mrr.query_id]
baselines_mrr.similarity_mrr=[1/Get_ranks(x, func=similarity_rec) for x in baselines_mrr.query_id]
baselines_mrr.flesch_mrr=[1/Get_ranks(x, func=Flesch_rec) for x in baselines_mrr.query_id]
baselines_mrr.Read2vec_mrr=[1/Get_ranks(x, func=Read2vec_rec) for x in baselines_mrr.query_id]
baselines_mrr.flesch_dist_mrr=[1/Get_ranks(x, func=Flesch_q_a_distance) for x in baselines_mrr.query_id]
baselines_mrr.Read2vec_dist_mrr=[1/Get_ranks(x, func=Read2vec_q_a_distance) for x in baselines_mrr.query_id]

In [101]:
baselines_mrr['Read2vec_inv']=[1/(Get_ranks(x, func=Read2vec_rec_inv)) for x in baselines_mrr.query_id]

In [102]:
baselines_mrr[:2]

,query_id,random_mrr,similarity_mrr,flesch_mrr,Read2vec_mrr,flesch_dist_mrr,Read2vec_dist_mrr,Read2vec_inv
0,1,0.166667,0.166667,1.000000,0.1,0.125,0.200000,0.500000
1,2,0.500000,0.333333,0.166667,1.0,0.250,0.111111,0.111111


In [103]:
baselines_mrr=baselines_mrr.replace([np.inf, -np.inf], np.nan)

In [104]:
average_ranks_mrr = pd.DataFrame(columns=('random','similarity','flesch','Read2vec','Read2vec_inv', 'flesch_dist','Read2vec_dist'))

In [105]:
average_ranks_mrr['random']= [baselines_mrr['random_mrr'].mean()]
average_ranks_mrr['similarity']= [baselines_mrr['similarity_mrr'].mean()]
average_ranks_mrr['flesch']= [baselines_mrr['flesch_mrr'].mean()]
average_ranks_mrr['Read2vec']= [baselines_mrr['Read2vec_mrr'].mean()]
average_ranks_mrr['Read2vec_inv']= [baselines_mrr['Read2vec_inv'].mean()]
average_ranks_mrr['flesch_dist']= [baselines_mrr['flesch_dist_mrr'].mean()]
average_ranks_mrr['Read2vec_dist']= [baselines_mrr['Read2vec_dist_mrr'].mean()]

In [106]:
average_ranks_mrr

,random,similarity,flesch,Read2vec,Read2vec_inv,flesch_dist,Read2vec_dist
0,0.486112,0.548105,0.444367,0.470044,0.463713,0.509728,0.464754


In [107]:
len(new_ids)

193

In [108]:
len(all_ids)

1571

In [109]:
freq= list(set([v for k,v in cnt.items()]))

In [110]:
freq

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18]

In [111]:
count=0
numbers=0
new_dict={}

for k,v in cnt.items():
    if v==11:        
        numbers=numbers+1
        new_dict[1]=numbers
        count=count+1

In [112]:
new_dict

{1: 17}

In [1161]:
def count_questions(nums=''):
    counts=0
    numbs=0
    numbers_dict={}    
    
    for k,v in cnt.items():
        if v==nums:
            numbs=numbs+1                       
            counts=counts+1
    numbers_dict[str(numbs)+' querie(s)']= str(nums)+' answer(s)'        
    return numbers_dict

In [1162]:
numbers_list=[]
for f in freq:
    numbers_list.append(count_questions(f).copy())

In [1163]:
numbers_list

[{'245 querie(s)': '1 answer(s)'},
 {'61 querie(s)': '2 answer(s)'},
 {'13 querie(s)': '3 answer(s)'},
 {'5 querie(s)': '4 answer(s)'},
 {'4 querie(s)': '5 answer(s)'},
 {'7 querie(s)': '6 answer(s)'},
 {'10 querie(s)': '7 answer(s)'},
 {'10 querie(s)': '8 answer(s)'},
 {'18 querie(s)': '9 answer(s)'},
 {'16 querie(s)': '10 answer(s)'},
 {'17 querie(s)': '11 answer(s)'},
 {'11 querie(s)': '12 answer(s)'},
 {'11 querie(s)': '13 answer(s)'},
 {'7 querie(s)': '14 answer(s)'},
 {'1 querie(s)': '16 answer(s)'},
 {'1 querie(s)': '17 answer(s)'},
 {'1 querie(s)': '18 answer(s)'}]

In [116]:
### Use different classifiers for the error rate calculation

In [117]:
logiregr = linear_model.LogisticRegression(C=1.0)

#### Using the combination of the similarity with flesch answer (Model 1)

In [118]:
mod1_x= train_data[['query_answer_sim','flesch_answer']]
mod1_y= train_data[['Relevance_score']]

logiregr.fit(mod1_x,mod1_y)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [119]:
mod1=test_data[['query_answer_sim','flesch_answer', 'Relevance_score']]

mod1_test=mod1[['query_answer_sim','flesch_answer']]

mod1_res=logiregr.predict(mod1_test)

mod1['pred']=mod1_res
mod1['error_rate']=[(x-y) for x,y in zip(mod1.pred,mod1.Relevance_score)]

mod1_error=[mod1['error_rate'].mean()]

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [120]:
mod1_error

[0.5852187028657617]

#### Using the combination of the similarity with flesch distance (Model 2)

In [121]:
logiregr2 = linear_model.LogisticRegression(C=1.0)

mod2_x= train_data[['query_answer_sim','flesch_dist']] #train with similarty and flesch distance
mod2_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr2.fit(mod2_x,mod2_y) #fit imto the model


mod2=test_data[['query_answer_sim','flesch_dist', 'Relevance_score']] #select some columns from the test set

mod2_test=mod2[['query_answer_sim','flesch_dist']] #select the data for prediction

mod2_res=logiregr2.predict(mod2_test) #predict the relevance scores for the second model

mod2['pred']=mod2_res #put the scores in the dataframe
mod2['error_rate']=[(x-y) for x,y in zip(mod2.pred,mod2.Relevance_score)] #calculate the error

mod2_error=[mod2['error_rate'].mean()] #get the mean of the error

print(mod2_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.7111613876319759]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with read2vec answer (Model 3)

In [122]:
logiregr3 = linear_model.LogisticRegression(C=1.0)

mod3_x= train_data[['query_answer_sim','Read2vec_answer']] #train with similarty and flesch distance
mod3_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr3.fit(mod3_x,mod3_y) #fit imto the model


mod3=test_data[['query_answer_sim','Read2vec_answer', 'Relevance_score']] #select some columns from the test set

mod3_test=mod3[['query_answer_sim','Read2vec_answer']] #select the data for prediction

mod3_res=logiregr3.predict(mod3_test) #predict the relevance scores for the second model

mod3['pred']=mod3_res #put the scores in the dataframe
mod3['error_rate']=[(x-y) for x,y in zip(mod3.pred,mod3.Relevance_score)] #calculate the error

mod3_error=[mod3['error_rate'].mean()] #get the mean of the error

print(mod3_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.6093514328808446]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with read2vec distance (Model 4)

In [123]:
logiregr4 = linear_model.LogisticRegression(C=1.0)

mod4_x= train_data[['query_answer_sim','read2vec_dist']] #train with similarty and flesch distance
mod4_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr4.fit(mod4_x,mod4_y) #fit imto the model


mod4=test_data[['query_answer_sim','read2vec_dist', 'Relevance_score']] #select some columns from the test set

mod4_test=mod4[['query_answer_sim','read2vec_dist']] #select the data for prediction

mod4_res=logiregr4.predict(mod4_test) #predict the relevance scores for the second model

mod4['pred']=mod4_res #put the scores in the dataframe
mod4['error_rate']=[(x-y) for x,y in zip(mod4.pred,mod4.Relevance_score)] #calculate the error

mod4_error=[mod4['error_rate'].mean()] #get the mean of the error

print(mod4_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.6327300150829562]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with flesch answer and flesch distance (Model 5)

In [124]:
logiregr5 = linear_model.LogisticRegression(C=1.0)

mod5_x= train_data[['query_answer_sim','flesch_answer','flesch_dist']] #train with similarty and flesch distance
mod5_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr5.fit(mod5_x,mod5_y) #fit imto the model


mod5=test_data[['query_answer_sim','flesch_answer','flesch_dist','Relevance_score']] #select some columns from the test set

mod5_test=mod5[['query_answer_sim','flesch_answer','flesch_dist']] #select the data for prediction

mod5_res=logiregr5.predict(mod5_test) #predict the relevance scores for the second model

mod5['pred']=mod5_res #put the scores in the dataframe
mod5['error_rate']=[(x-y) for x,y in zip(mod5.pred,mod5.Relevance_score)] #calculate the error

mod5_error=[mod5['error_rate'].mean()] #get the mean of the error

print(mod5_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.5844645550527904]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with read2vec answer and read2vec distance (Model 6)

In [125]:
logiregr6 = linear_model.LogisticRegression(C=1.0)

mod6_x= train_data[['query_answer_sim','Read2vec_answer','read2vec_dist']] #train with similarty and flesch distance
mod6_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr6.fit(mod6_x,mod6_y) #fit imto the model


mod6=test_data[['query_answer_sim','Read2vec_answer','read2vec_dist','Relevance_score']] #select some columns from the test set

mod6_test=mod6[['query_answer_sim','Read2vec_answer','read2vec_dist']] #select the data for prediction

mod6_res=logiregr6.predict(mod6_test) #predict the relevance scores for the second model

mod6['pred']=mod6_res #put the scores in the dataframe
mod6['error_rate']=[(x-y) for x,y in zip(mod6.pred,mod6.Relevance_score)] #calculate the error

mod6_error=[mod6['error_rate'].mean()] #get the mean of the error

print(mod6_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.5746606334841629]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with flesch answer and read2vec distance (Model 7)

In [126]:
logiregr7 = linear_model.LogisticRegression(C=1.0)

mod7_x= train_data[['query_answer_sim','flesch_answer','read2vec_dist']] #train with similarty and flesch distance
mod7_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr7.fit(mod7_x,mod7_y) #fit imto the model


mod7=test_data[['query_answer_sim','flesch_answer','read2vec_dist','Relevance_score']] #select some columns from the test set

mod7_test=mod7[['query_answer_sim','flesch_answer','read2vec_dist']] #select the data for prediction

mod7_res=logiregr7.predict(mod7_test) #predict the relevance scores for the second model

mod7['pred']=mod7_res #put the scores in the dataframe
mod7['error_rate']=[(x-y) for x,y in zip(mod7.pred,mod7.Relevance_score)] #calculate the error

mod7_error=[mod7['error_rate'].mean()] #get the mean of the error

print(mod7_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.5414781297134238]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with read2vec answer and flesch distance (Model 8)

In [127]:
logiregr8 = linear_model.LogisticRegression(C=1.0)

mod8_x= train_data[['query_answer_sim','Read2vec_answer','flesch_dist']] #train with similarty and flesch distance
mod8_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr8.fit(mod8_x,mod8_y) #fit imto the model


mod8=test_data[['query_answer_sim','Read2vec_answer','flesch_dist','Relevance_score']] #select some columns from the test set

mod8_test=mod8[['query_answer_sim','Read2vec_answer','flesch_dist']] #select the data for prediction

mod8_res=logiregr8.predict(mod8_test) #predict the relevance scores for the second model

mod8['pred']=mod8_res #put the scores in the dataframe
mod8['error_rate']=[(x-y) for x,y in zip(mod8.pred,mod8.Relevance_score)] #calculate the error

mod8_error=[mod8['error_rate'].mean()] #get the mean of the error

print(mod8_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.6131221719457014]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with flesch ans, read2vec answer and read2vec distance (Model 9)

In [1273]:
logiregr9 = linear_model.LogisticRegression(C=1.0)

mod9_x= train_data[['query_answer_sim','flesch_answer','Read2vec_answer','read2vec_dist']] #train with similarty and flesch distance
mod9_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr9.fit(mod9_x,mod9_y) #fit imto the model


mod9=test_data[['query_answer_sim','flesch_answer','Read2vec_answer','read2vec_dist','Relevance_score']] #select some columns from the test set

mod9_test=mod9[['query_answer_sim','flesch_answer','Read2vec_answer','read2vec_dist']] #select the data for prediction

mod9_res=logiregr9.predict(mod9_test) #predict the relevance scores for the second model

mod9['pred']=mod9_res #put the scores in the dataframe
mod9['error_rate']=[(x-y) for x,y in zip(mod9.pred,mod9.Relevance_score)] #calculate the error

mod9_error=[mod9['error_rate'].mean()] #get the mean of the error

print(mod9_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.526395173453997]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Using the combination of the similarity with read2vec ans, flesch answer and flesch distance (Model 10)

In [129]:
logiregr10 = linear_model.LogisticRegression(C=1.0)

mod10_x= train_data[['query_answer_sim','flesch_answer','Read2vec_answer','flesch_dist']] #train with similarty and flesch distance
mod10_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr10.fit(mod10_x,mod10_y) #fit imto the model


mod10=test_data[['query_answer_sim','flesch_answer','Read2vec_answer','flesch_dist','Relevance_score']] #select some columns from the test set

mod10_test=mod10[['query_answer_sim','flesch_answer','Read2vec_answer','flesch_dist']] #select the data for prediction

mod10_res=logiregr10.predict(mod10_test) #predict the relevance scores for the second model

mod10['pred']=mod10_res #put the scores in the dataframe
mod10['error_rate']=[(x-y) for x,y in zip(mod10.pred,mod10.Relevance_score)] #calculate the error

mod10_error=[mod10['error_rate'].mean()] #get the mean of the error

print(mod10_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.5535444947209653]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1328]:
logiregr11 = linear_model.LogisticRegression(C=1.0)

mod11_x= train_data[['query_answer_sim','query_question_sim','read2vec_dist']] #train with similarty and flesch distance
mod11_y= train_data[['Relevance_score']] #fit it with the relevance score

logiregr11.fit(mod11_x,mod11_y) #fit imto the model


mod11=test_data[['query_answer_sim','flesch_answer','Read2vec_answer','flesch_dist','read2vec_dist','query_question_sim','Relevance_score']] #select some columns from the test set

mod11_test=mod11[['query_answer_sim','query_question_sim','read2vec_dist']] #select the data for prediction

mod11_res=logiregr11.predict(mod11_test) #predict the relevance scores for the second model

mod11['pred']=mod11_res #put the scores in the dataframe
mod11['error_rate']=[(x-y) for x,y in zip(mod11.pred,mod11.Relevance_score)] #calculate the error

mod11_error=[mod11['error_rate'].mean()] #get the mean of the error

print(mod11_error)

/var/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.6244343891402715]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1498]:
lm11.intercept_

array([ 2.26636207])

In [1497]:
lm11.coef_

array([[ 0.58113814,  0.20465715,  0.11752131]])

##### The model 7 returned the lowest error, as a result of the combination, so we will use this instead for the classification

In [1330]:
def pipeline(q_id=''):  
   
    query_id=int(q_id)   
    
    own=test_data[test_data['query_id']==query_id]
    own=own[['query_id','queries','question_content','query_question_sim','answer','query_answer_sim','Read2vec_answer','flesch_answer','read2vec_dist','Relevance_score']]
    own_tx=own[['query_answer_sim','query_question_sim','read2vec_dist']]   
    own_res=lm11.predict(own_tx) #predict the relevance scores for the second model
    own['pred']=own_res #put the scores in the dataframe  
    own.pred=[int(round(d)) for d in own.pred]

    ideal= own.sort_values('Relevance_score', ascending=False) #sorts the dataframe using the satisfaction
    ideal=ideal['answer'][:1] #the one with the lowest sat_score is the most relevant, select the first answer
    ideal=' '.join(str(s) for s in ideal) #convert answer to string
    own_rank= own.sort_values('pred', ascending=False) #firstly rank the selected dataframe by the similarity
    
    Recs=[s for s in own_rank.answer] #The recommendations are the answers based on the rank
    
    position= Recs.index(ideal)+1 #check where the relevant answer was ranked among the list of recommendations
    #own_rank['MRR']=1/position
    return position

In [1331]:
pipeline(q_id=593)

1

In [1332]:
New_Mrr=pd.DataFrame(columns=('query_id','Combination'))

In [1334]:
New_Mrr.query_id=new_ids

In [1336]:
New_Mrr.Combination=[1/pipeline(s) for s in New_Mrr.query_id]

In [ ]:
Get_ranks('593', func=random_rec)

In [1492]:
New_Mrr['Query_Ques_sim']=[1/Get_ranks(s, func=similarity_rec) for s in New_Mrr.query_id]

In [1493]:
Average_new_mrr= [New_Mrr['Query_Ques_sim'].mean()]

In [1494]:
Average_new_mrr

[0.4488318819925036]

In [1345]:
Average_Mrr=pd.DataFrame(columns=('Combination','Random','Flesch_Answer','Read2Vec_distance','Similarity'))

In [1350]:
Average_Mrr.Similarity= [New_Mrr['Similarity'].mean()]

In [1353]:
New_Mrr.head()

,query_id,Combination,Read2Vec_dist,Random,Similarity,Flesch_Answer
0,1,0.125,0.200000,0.500000,0.166667,1.000000
1,2,0.500,0.111111,0.250000,0.333333,0.166667
2,3,0.500,0.166667,0.142857,1.000000,0.500000
3,4,0.250,0.500000,0.250000,0.166667,1.000000
4,5,1.000,0.250000,0.100000,0.166667,0.111111


In [1351]:
Average_Mrr

,Combination,Random,Flesch_Answer,Read2Vec_distance,Similarity
0,0.707772,0.439479,0.444367,0.464754,0.548105


In [1326]:
with open('data/Mrr.csv','w') as f:
    print (New_Mrr.to_csv(sep=',', index=False, header=True), file=f)

pipeline('593',func='flesch_answer',a=0.3,b=0.7)

In [ ]:
# Combining content similarity with flesch answer/flesch distance pair or read2vec answer/read2vec distance pair

In [380]:
def pipelines2(q_id='',func1='', func2='', a='',b='',c=''):
    a=float(a) #datapoint a
    b=float(b) #datapoint b
    c=float(c)
    x=func1 #I can call any function based on the readability e.g Flesch
    y=func2 #for the distance
    #sim(qanda) and readabilityofa and distance
    query_id=int(q_id) #The query id is recognized as a string, so i convert to an integer
    own=new_data[new_data['query_id']==query_id] #select only rows that corresponds to the query_id entered   
    own=own[['query_id','queries','question_content','answer','query_ans_sat','query_answer_sim',x,y]] #select relevant columns
    ideal= own.sort_values('query_ans_sat', ascending=True) #sorts the dataframe using the satisfaction
    ideal=ideal['answer'][:1] #the one with the lowest sat_score is the most relevant, select the first answer
    ideal=' '.join(str(s) for s in ideal) #convert answer to string
    own_rank= own.sort_values('query_answer_sim', ascending=False) #firstly rank the selected dataframe by the similarity
    if x=='flesch_answer' or y=='flesch_answer':        
        own_rank['query_answer_sim']=[100* x for x in own_rank.query_answer_sim] #normalize the similarity with the readability
    elif x=='q_a_flesch_dist' or y=='q_a_flesch_dist':
        own_rank['query_answer_sim']=[100* x for x in own_rank.query_answer_sim]
    else:
        own_rank['query_answer_sim']=[x for x in own_rank.query_answer_sim] #normalize the similarity with the readability
    own_rank['data_point']=[(a*d)+(b*e)+(c*f) for d,e,f in zip(own_rank.query_answer_sim, own_rank[x], own_rank[y])] #check for the best datapoint
    own_rank= own_rank.sort_values('data_point', ascending=False) #rank based on the highest datapoint value
    Recs=[s for s in own_rank.answer] #The recommendations are the answers based on the rank
    
    position= Recs.index(ideal)+1    #check where the relevant answer was ranked among the list of recommendations
    return 1/position #return the rank

In [138]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [153]:
lm1 = LinearRegression()

lm1.fit(mod1_x,mod1_y) #fit imto the model

mod1_lm_res=lm1.predict(mod1_test) #predict the relevance scores for the second model

mod1['lm_pred']= mod1_lm_res#put the scores in the dataframe
mod1.lm_pred=[int(round(d)) for d in mod1.lm_pred]
mod1['lm_error_rate']=[(x-y) for x,y in zip(mod1.lm_pred,mod1.Relevance_score)] #calculate the error

mod1_lm_error=[mod1['lm_error_rate'].mean()] #get the mean of the error

print(mod1_lm_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.39819004524886875]


/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [660]:
print(lm10.intercept_)
print(lm10.coef_)

[ 2.76695311]
[[ 0.65762819 -0.00427898 -0.11772302 -0.00138803]]


In [155]:
lm2 = LinearRegression()

lm2.fit(mod2_x,mod2_y) #fit imto the model

mod2_lm_res=lm2.predict(mod2_test) #predict the relevance scores for the second model

mod2['lm_pred']=mod2_lm_res #put the scores in the dataframe
mod2.lm_pred=[int(round(d)) for d in mod2.lm_pred]
mod2['lm_error_rate']=[(x-y) for x,y in zip(mod2.lm_pred,mod2.Relevance_score)] #calculate the error

mod2_lm_error=[mod2['lm_error_rate'].mean()] #get the mean of the error

print(mod2_lm_error)

[0.4276018099547511]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [157]:
lm3 = LinearRegression()

lm3.fit(mod3_x,mod3_y) #fit imto the model

mod3_lm_res=lm3.predict(mod3_test) #predict the relevance scores for the second model

mod3['lm_pred']=mod3_lm_res #put the scores in the dataframe
mod3.lm_pred=[int(round(d)) for d in mod3.lm_pred]
mod3['lm_error_rate']=[(x-y) for x,y in zip(mod3.lm_pred,mod3.Relevance_score)] #calculate the error

mod3_lm_error=[mod3['lm_error_rate'].mean()] #get the mean of the error

print(mod3_lm_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


[0.4079939668174962]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [158]:
lm4 = LinearRegression()

lm4.fit(mod4_x,mod4_y) #fit imto the model

mod4_lm_res=lm4.predict(mod4_test) #predict the relevance scores for the second model

mod4['lm_pred']=mod4_lm_res #put the scores in the dataframe
mod4.lm_pred=[int(round(d)) for d in mod4.lm_pred]
mod4['lm_error_rate']=[(x-y) for x,y in zip(mod4.lm_pred,mod4.Relevance_score)] #calculate the error

mod4_lm_error=[mod4['lm_error_rate'].mean()] #get the mean of the error

print(mod4_lm_error)

[0.43288084464555054]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [159]:
lm5 = LinearRegression()

lm5.fit(mod5_x,mod5_y) #fit imto the model

mod5_lm_res=lm5.predict(mod5_test) #predict the relevance scores for the second model

mod5['lm_pred']=mod5_lm_res #put the scores in the dataframe
mod5.lm_pred=[int(round(d)) for d in mod5.lm_pred]
mod5['lm_error_rate']=[(x-y) for x,y in zip(mod5.lm_pred,mod5.Relevance_score)] #calculate the error

mod5_lm_error=[mod5['lm_error_rate'].mean()] #get the mean of the error

print(mod5_lm_error)

[0.4034690799396682]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [160]:
lm6 = LinearRegression()

lm6.fit(mod6_x,mod6_y) #fit imto the model

mod6_lm_res=lm6.predict(mod6_test) #predict the relevance scores for the second model

mod6['lm_pred']=mod6_lm_res #put the scores in the dataframe
mod6.lm_pred=[int(round(d)) for d in mod6.lm_pred]
mod6['lm_error_rate']=[(x-y) for x,y in zip(mod6.lm_pred,mod6.Relevance_score)] #calculate the error

mod6_lm_error=[mod6['lm_error_rate'].mean()] #get the mean of the error

print(mod6_lm_error)

[0.4155354449472097]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [161]:
lm7 = LinearRegression()

lm7.fit(mod7_x,mod7_y) #fit imto the model

mod7_lm_res=lm7.predict(mod7_test) #predict the relevance scores for the second model

mod7['lm_pred']=mod7_lm_res #put the scores in the dataframe
mod7.lm_pred=[int(round(d)) for d in mod7.lm_pred]

mod7['lm_error_rate']=[(x-y) for x,y in zip(mod7.lm_pred,mod7.Relevance_score)] #calculate the error

mod7_lm_error=[mod7['lm_error_rate'].mean()] #get the mean of the error

print(mod7_lm_error)

[0.4004524886877828]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [162]:
lm8 = LinearRegression()

lm8.fit(mod8_x,mod8_y) #fit imto the model

mod8_lm_res=lm8.predict(mod8_test) #predict the relevance scores for the second model

mod8['lm_pred']=mod8_lm_res #put the scores in the dataframe
mod8.lm_pred=[int(round(d)) for d in mod8.lm_pred]
mod8['lm_error_rate']=[(x-y) for x,y in zip(mod8.lm_pred,mod8.Relevance_score)] #calculate the error

mod8_lm_error=[mod8['lm_error_rate'].mean()] #get the mean of the error

print(mod8_lm_error)

[0.40497737556561086]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [163]:
lm9 = LinearRegression()

lm9.fit(mod9_x,mod9_y) #fit imto the model

mod9_lm_res=lm9.predict(mod9_test) #predict the relevance scores for the second model

mod9['lm_pred']=mod9_lm_res #put the scores in the dataframe
mod9.lm_pred=[int(round(d)) for d in mod9.lm_pred]

mod9['lm_error_rate']=[(x-y) for x,y in zip(mod9.lm_pred,mod9.Relevance_score)] #calculate the error

mod9_lm_error=[mod9['lm_error_rate'].mean()] #get the mean of the error

print(mod9_lm_error)

[0.39441930618401205]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [164]:
lm10 = LinearRegression()

lm10.fit(mod10_x,mod10_y) #fit imto the model

mod10_lm_res=lm10.predict(mod10_test) #predict the relevance scores for the second model

mod10['lm_pred']=mod10_lm_res #put the scores in the dataframe
mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod10['lm_error_rate']=[(x-y) for x,y in zip(mod10.lm_pred,mod10.Relevance_score)] #calculate the error

mod10_lm_error=[mod10['lm_error_rate'].mean()] #get the mean of the error

print(mod10_lm_error)

[0.39668174962292607]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [1329]:
lm11 = LinearRegression()

lm11.fit(mod11_x,mod11_y) #fit imto the model

mod11_lm_res=lm11.predict(mod11_test) #predict the relevance scores for the second model

mod11['lm_pred']=mod11_lm_res #put the scores in the dataframe
mod11.lm_pred=[int(round(d)) for d in mod11.lm_pred]
mod11['lm_error_rate']=[(x-y) for x,y in zip(mod11.lm_pred,mod11.Relevance_score)] #calculate the error

mod11_lm_error=[mod11['lm_error_rate'].mean()] #get the mean of the error

print(mod11_lm_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.42232277526395173]


/var/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Using random Forest

In [187]:
from sklearn.ensemble import RandomForestClassifier

In [189]:
rfc1 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc1.fit(mod10_x,mod10_y) #fit imto the model

mod10_rf_res=np.array(rfc1.predict(mod10_test))#predict the relevance scores for the second model

mod10['rf_pred']=mod10_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod10['rf_error_rate']=[(x-y) for x,y in zip(mod10.rf_pred,mod10.Relevance_score)] #calculate the error

mod10_rf_error=[mod10['rf_error_rate'].mean()] #get the mean of the error

print(mod10_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.4381598793363499]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [191]:
rfc11 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc11.fit(mod1_x,mod1_y) #fit imto the model

mod1_rf_res=np.array(rfc11.predict(mod1_test))#predict the relevance scores for the second model

mod1['rf_pred']=mod1_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod1['rf_error_rate']=[(x-y) for x,y in zip(mod1.rf_pred,mod1.Relevance_score)] #calculate the error

mod1_rf_error=[mod1['rf_error_rate'].mean()] #get the mean of the error

print(mod1_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.41855203619909503]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [192]:
rfc2 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc2.fit(mod2_x,mod2_y) #fit imto the model

mod2_rf_res=np.array(rfc2.predict(mod2_test))#predict the relevance scores for the second model

mod2['rf_pred']=mod2_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod2['rf_error_rate']=[(x-y) for x,y in zip(mod2.rf_pred,mod2.Relevance_score)] #calculate the error

mod2_rf_error=[mod2['rf_error_rate'].mean()] #get the mean of the error

print(mod2_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.46907993966817496]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [193]:
rfc3 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc3.fit(mod3_x,mod3_y) #fit imto the model

mod3_rf_res=np.array(rfc3.predict(mod3_test))#predict the relevance scores for the second model

mod3['rf_pred']=mod3_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod3['rf_error_rate']=[(x-y) for x,y in zip(mod3.rf_pred,mod3.Relevance_score)] #calculate the error

mod3_rf_error=[mod3['rf_error_rate'].mean()] #get the mean of the error

print(mod3_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.43891402714932126]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [194]:
rfc4 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc4.fit(mod4_x,mod4_y) #fit imto the model

mod4_rf_res=np.array(rfc4.predict(mod4_test))#predict the relevance scores for the second model

mod4['rf_pred']=mod4_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod4['rf_error_rate']=[(x-y) for x,y in zip(mod4.rf_pred,mod4.Relevance_score)] #calculate the error

mod4_rf_error=[mod4['rf_error_rate'].mean()] #get the mean of the error

print(mod4_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.40271493212669685]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [195]:
rfc5 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc5.fit(mod5_x,mod5_y) #fit imto the model

mod5_rf_res=np.array(rfc5.predict(mod5_test))#predict the relevance scores for the second model

mod5['rf_pred']=mod5_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod5['rf_error_rate']=[(x-y) for x,y in zip(mod5.rf_pred,mod5.Relevance_score)] #calculate the error

mod5_rf_error=[mod5['rf_error_rate'].mean()] #get the mean of the error

print(mod5_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.42458521870286575]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [196]:
rfc6 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc6.fit(mod6_x,mod6_y) #fit imto the model

mod6_rf_res=np.array(rfc6.predict(mod6_test))#predict the relevance scores for the second model

mod6['rf_pred']=mod6_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod6['rf_error_rate']=[(x-y) for x,y in zip(mod6.rf_pred,mod6.Relevance_score)] #calculate the error

mod6_rf_error=[mod6['rf_error_rate'].mean()] #get the mean of the error

print(mod6_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.41855203619909503]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [197]:
rfc7 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc7.fit(mod7_x,mod7_y) #fit imto the model

mod7_rf_res=np.array(rfc7.predict(mod7_test))#predict the relevance scores for the second model

mod7['rf_pred']=mod7_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod7['rf_error_rate']=[(x-y) for x,y in zip(mod7.rf_pred,mod7.Relevance_score)] #calculate the error

mod7_rf_error=[mod7['rf_error_rate'].mean()] #get the mean of the error

print(mod7_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.45098039215686275]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [198]:
rfc8 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc8.fit(mod8_x,mod8_y) #fit imto the model

mod8_rf_res=np.array(rfc8.predict(mod8_test))#predict the relevance scores for the second model

mod8['rf_pred']=mod8_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod8['rf_error_rate']=[(x-y) for x,y in zip(mod8.rf_pred,mod8.Relevance_score)] #calculate the error

mod8_rf_error=[mod8['rf_error_rate'].mean()] #get the mean of the error

print(mod8_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.4517345399698341]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [199]:
rfc9 = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rfc9.fit(mod9_x,mod9_y) #fit imto the model

mod9_rf_res=np.array(rfc9.predict(mod9_test))#predict the relevance scores for the second model

mod9['rf_pred']=mod9_rf_res #put the scores in the dataframe
#mod10.lm_pred=[int(round(d)) for d in mod10.lm_pred]
mod9['rf_error_rate']=[(x-y) for x,y in zip(mod9.rf_pred,mod9.Relevance_score)] #calculate the error

mod9_rf_error=[mod9['rf_error_rate'].mean()] #get the mean of the error

print(mod9_rf_error)

/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


[0.4321266968325792]


/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/var/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [676]:
def NDCG_scores(q_id=''):
    que_id=int(q_id)
    recs=similarity_rec(que_id)
    own= test_data[test_data['query_id']==que_id]    
    ideal= own.sort_values('Relevance_score', ascending=False)
    ideal_list=[s for s in ideal.Relevance_score] #ideal rank of the items. this list is sorted
    ideal_answers=[s for s in ideal.answer] #these are the list of answers when the dataframe is sorted by relevance
    #unsorted_list =[s for s in own.Relevance_score] #where it initially ranked the items 
    #sorted_ideal_list=sorted(ideal_list, reverse=True)
    #get the position that the recommenations have been ranked based on the function used          
    
    function_ranks=[] #ranks based on the function used. Each function produces a list of recs, get where those recs were ranked based on relevance
    for n in recs:    
        function_ranks.append([s for s in own['Relevance_score'][own['answer']==n]]) #Relevance scores of each recommended answer
    function_ranks=[item for sublist in function_ranks for item in sublist] #stores the relevance scores of the recommendations produced
    
    ranks=[i+1 for i in range(0, len(ideal_list))] #this is for the ranks based on the number of recommendations on the list
    rank_logs=[round(math.log2(i+1),3) for i in ranks]
    dcg= sum([x/y for x,y in zip(function_ranks,rank_logs)])
    idcg= sum([x/y for x,y in zip(ideal_list, rank_logs)])
    ndcg=(dcg/idcg)  
    #print(ideal_list)
    #print(function_ranks)
    #print(len(own))
    return ndcg
    #return idcg

In [678]:
NDCG_scores('592')

0.9849094162816453

In [1355]:
def Candidates(q_id=''):  
   
    query_id=int(q_id)   
    
    own=test_data[test_data['query_id']==query_id]
    own=own[['query_id','queries','query_question_sim','answer','query_answer_sim','Read2vec_answer','flesch_answer','read2vec_dist','Relevance_score']]
    own_tx=own[['query_answer_sim','query_question_sim','read2vec_dist']]    
    own_res=lm11.predict(own_tx) #predict the relevance scores for the second model
    own['pred']=own_res #put the scores in the dataframe  
    own.pred=[int(round(d)) for d in own.pred]


    ideal= own.sort_values('Relevance_score', ascending=False) #sorts the dataframe using the satisfaction
    ideal=ideal['answer'][:1] #the one with the lowest sat_score is the most relevant, select the first answer
    ideal=' '.join(str(s) for s in ideal) #convert answer to string
    own_rank= own.sort_values('pred', ascending=False) #firstly rank the selected dataframe by the similarity
    
    Recs=[s for s in own_rank.answer] #The recommendations are the answers based on the rank
    
    position= Recs.index(ideal)+1 #check where the relevant answer was ranked among the list of recommendations
    #own_rank['MRR']=1/position
    #print(position)
    return Recs

In [1356]:
Candidates(593)

['1 cc of water = 1 gram\t\t2 liters = 2000 cc = 2000 grams = 2 Kg',
 'Firstly, calculate the capacity of the tank in cubic cm.\t\t25 x 10 x 8 = 2,000 cubic cm\t\tThen convert cubic cm (or "cc") to liters: 1,000 cc = 1 litre\t\tTherefore the tank holds 2 liters of water.\t\t–––––––––––––––––––––––––––',
 'It means the amount of empty space (how much liquid it would hold) in a motorcycle engine when the piston is at the bottom of the stroke.  For instance a chevy 350 Cubic  Inch motor holds 350 cubic inches of liquid.  To compare - a 1600cc holds approx. 95 Cubic Inches of liquid.',
 'Only Mazda offered rotary engines, Toyota did not. Mazda still has the RX-8 with a rotary engine.']

In [1393]:
nd_test=pd.DataFrame(columns=("q_id","Combination","Random","Similarity","Flesch_Answer","Read2Vec_distance"))

In [1486]:
nd_test2=pd.DataFrame(columns=("q_id","Combination"))

In [1487]:
nd_test2.q_id=new_ids

In [1488]:
nd_test2.Combination=[Lentor_scores(x) for x in nd_test2.q_id]

In [1489]:
nd_test2.head()

,q_id,Combination
0,1,0.669117
1,2,1.000000
2,3,0.821197
3,4,0.822047
4,5,0.780260


In [1490]:
Candidates_ndcg=[nd_test2['Combination'].mean()]

In [1491]:
Candidates_ndcg

[0.8285213417016153]

In [1411]:
Average_Ndcg=pd.DataFrame(columns=('Combination','Random','Flesch_Answer','Read2Vec_distance','Similarity'))

In [1417]:
Average_Ndcg

,Combination,Random,Flesch_Answer,Read2Vec_distance,Similarity
0,0.868998,0.832152,0.821687,0.829855,0.861569


In [397]:
import letor_metrics

In [1043]:
letor_metrics.ndcg_score([3, 1],[1, 1])

0.70980974139686548

In [1454]:
def Lentor_scores(q_id=''):
    que_id=int(q_id)
    recs=similarity_rec(que_id)
    own= test_data[test_data['query_id']==que_id]    
    ideal= own.sort_values('Relevance_score', ascending=False)
    ideal_list=[s for s in ideal.Relevance_score]#ideal rank of the items. this list is sorted
    ideal_answers=[s for s in ideal.answer] #these are the list of answers when the dataframe is sorted by relevance
    #unsorted_list =[s for s in own.Relevance_score] #where it initially ranked the items 
    #sorted_ideal_list=sorted(ideal_list, reverse=True)
    #get the position that the recommenations have been ranked based on the function used          
    
    function_ranks=[] #ranks based on the function used. Each function produces a list of recs, get where those recs were ranked based on relevance
    for n in recs:    
        function_ranks.append([s for s in own['Relevance_score'][own['answer']==n]]) #Relevance scores of each recommended answer
    function_ranks=[item for sublist in function_ranks for item in sublist][:len(ideal_list)] #stores the relevance scores of the recommendations produced
   
    #print(function_ranks)
    #print(ideal_list)
    ndcg= letor_metrics.ndcg_score(ideal_list,function_ranks)
    
    return ndcg   #return idcg

In [1455]:
Lentor_scores(592)

0.85860399650272057

In [729]:
#from collections import Counter

#cnt= Counter(all_ids)
#new_ids= [k for k,v in cnt.items() if v>1] #saves query ids that have more than one answer to the question
thir_ans_ids= [k for k,v in cnt.items() if v==13]

In [1045]:
four_ans_ids

[60, 210, 292, 494, 593]

In [679]:
two_data= test_data[test_data.query_id.isin(two_ans_ids)]

In [1134]:
all_sents=[tokenizer.tokenize(str(s)) for s in data.answer]

In [1135]:
all_sent=[item for sublist in all_sents for item in sublist]

In [1136]:
len(all_sent)

9210

In [1137]:
sen_len=[len(s) for s in all_sents]

In [1138]:
tt=[s for s in data.answer]

In [1139]:
min(sen_len)

1

In [1140]:
maxes=list(set([s for s in sen_len if s>20]))

In [1141]:
long_s=[s for s in all_sents if len(s)==161]

In [1142]:
long_s=[item for sublist in long_s for item in sublist]

In [1104]:
test_data[60:65]

,query_id,queries,question_subject,question_content,answer,query_ans_sat,flesch_query,flesch_answer,Read2vec_query,Read2vec_answer,flesch_dist,read2vec_dist,Relevance_score,query_answer_sim
70,602,Which of the following best describes the diff...,Dose anyone one know anything about The Role o...,Which of the following best describes the diff...,Nuclear families are smaller.\t\tI think the a...,1.333333,46.78,64.10,0.000428,0.975334,17.32,0.974907,3,0.338033
71,602,Which of the following best describes the diff...,Which of the following best describes the diff...,\t,I don't see the choices. Nuclear families died...,1.428571,46.78,61.97,0.000428,0.246764,15.19,0.246336,3,0.267321
75,105,action reggae song,"Old Reggae song, chorus ""i need some action, t...",\t,Artist - Banton Buju\tSong - Action,1.416667,76.89,66.40,0.960182,0.959425,10.49,0.000757,3,0.163056
76,105,action reggae song,"WHO SINGS THAT 90'S REGGAE SONG ""ACTION""?","It's like... Action, by a girl, and then a guy...",Buju Banton,1.500000,76.89,52.53,0.960182,0.628398,24.36,0.331784,2,0.000000
77,555,add limewire songs to itunes automatically,How do I transfer music from LimeWire to ITune...,\t,on your Limewire \tgo to preferences \tThe opt...,1.428571,0.00,55.74,0.917372,0.999130,55.74,0.081758,3,0.363077


In [ ]:
Which of the following best describes the difference between an extended family and a nuclear family'\n 'Which of the following best describes the difference between an extended family and a nuclear family

In [1154]:
#504

In [1164]:
len(cnt)

438

In [1193]:
numbers_list

[{'245 querie(s)': '1 answer(s)'},
 {'61 querie(s)': '2 answer(s)'},
 {'13 querie(s)': '3 answer(s)'},
 {'5 querie(s)': '4 answer(s)'},
 {'4 querie(s)': '5 answer(s)'},
 {'7 querie(s)': '6 answer(s)'},
 {'10 querie(s)': '7 answer(s)'},
 {'10 querie(s)': '8 answer(s)'},
 {'18 querie(s)': '9 answer(s)'},
 {'16 querie(s)': '10 answer(s)'},
 {'17 querie(s)': '11 answer(s)'},
 {'11 querie(s)': '12 answer(s)'},
 {'11 querie(s)': '13 answer(s)'},
 {'7 querie(s)': '14 answer(s)'},
 {'1 querie(s)': '16 answer(s)'},
 {'1 querie(s)': '17 answer(s)'},
 {'1 querie(s)': '18 answer(s)'}]

In [1195]:
len(test_data)

1326